In [ ]:
import numpy as np
from utils.dataloader import *
from utils.coordinate_calibration import PointCloudRegistration
from utils.meshgrid import Meshgrid
from utils.icp import IterativeClosestPoint, Matching
import os
from test import FileOutputMatcher

In [ ]:
OUTPUT_DIR = './OUTPUT'
DATA_DIR = './pa345_data'

RIGID_BODY_DATA_A = f'{DATA_DIR}/Problem3-BodyA.txt'
RIGID_BODY_DATA_B = f'{DATA_DIR}/Problem3-BodyB.txt'

SURFACE_DATA = f'{DATA_DIR}/Problem3MeshFile.sur'

SAMPLE_ID = ''
SAMPLE_DATA = f'./pa345_data/PA3-{SAMPLE_ID}-Debug-SampleReadingsTest.txt'
# SAMPLE_DATA = f'./pa345_data/PA3-{SAMPLE_ID}-Unknown-SampleReadingsTest.txt'

# Load data files
rigidbody_dl_A = RigidBodyDataloader.read_file(RIGID_BODY_DATA_A)
rigidbody_dl_B = RigidBodyDataloader.read_file(RIGID_BODY_DATA_B)

surface_dl = Surfaceloader.read_file(SURFACE_DATA)
sample_dl = SampleReadingsDataloader.read_file(SAMPLE_DATA, delimiter=',', N_A=rigidbody_dl_A.N_markers, N_B=rigidbody_dl_B.N_markers)

### Get F<sub>A, k</sub> and F<sub>B, k</sub>

In [ ]:
rigidbody_dl_A_markers = rigidbody_dl_A.markers.reshape(1, -1, 3) # markers of body A in body A coordinates

sample_dl_A = sample_dl.body_A # samples of body A markers in tracker coordinates
num_samples = sample_dl.N_samps

# perform registration for each frame
reg = PointCloudRegistration()
F_A = []
for i in range(num_samples):
    sample_dl_A_i = sample_dl_A[i].reshape(1, -1, 3)
    F_A_i, err = reg.register(rigidbody_dl_A_markers, sample_dl_A_i)
    F_A.append(F_A_i)

F_A = np.array(F_A)

ridigbody_dl_B_markers = rigidbody_dl_B.markers.reshape(1, -1, 3) # markers of body B in body B coordinates
sample_dl_B = sample_dl.body_B # samples of body B markers in tracker coordinates

# perform registration for each frame
F_B = []
for i in range(num_samples):
    sample_dl_B_i = sample_dl_B[i].reshape(1, -1, 3)

    F_B_i, err = reg.register(ridigbody_dl_B_markers, sample_dl_B_i)
    F_B.append(F_B_i)

F_B = np.array(F_B)

### Get d<sub>k

In [ ]:
A_tip = rigidbody_dl_A.tip
A_tip = np.append(A_tip, 1) # add 1 for homogenous coordinates
d_k = []

for i in range(num_samples):
    F_A_i = F_A[i] # get F_A for frame i
    F_B_i_inv = np.linalg.inv(F_B[i]) # get F_B inverse for frame i

    d_k.append(F_B_i_inv @ F_A_i @ A_tip) # d_k = F_B^-1 * F_A * A_tip

d_k = np.array(d_k)[:,:3]
print(d_k.shape)
print(d_k)

In [ ]:
import csv
import time

# Select algorithm for finding closest point
# Choices: SIMPLE_LINEAR, VECTORIZED_LINEAR, SIMPLE_OCTREE, VECTORIZED_OCTREE
matching_algo = Matching.VECTORIZED_OCTREE  # VECTORIZED_LINEAR is the fastest

# Initialize ICP helper class 
icp = IterativeClosestPoint(matching_algo)

# Initialize meshgrid of Triangles
mesh = Meshgrid(surface_dl.vertices, surface_dl.triangles)

# Find closest points and euclidean distances
elapsed_times = []

for i in range(5):
    start_time = time.time()
    pt, dist = icp.match(d_k, mesh)
    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_times.append(elapsed_time)
    print(f'Elapsed time: {elapsed_time:.2f}s')

# Write elapsed times to CSV
elapsed_time_csv = f'{OUTPUT_DIR}/elapsed_times.csv'
with open(elapsed_time_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Elapsed Times (s)'])
    writer.writerow([round(elapsed_time, 2) for elapsed_time in elapsed_times])

# pt, dist = icp.match(d_k, mesh)

print(f'Elapsed times written to {elapsed_time_csv}')

print(f'Closest points: {np.around(pt, 2)}')
print(f'Closest distances: {np.around(dist, 3)}')

### write output to file

In [ ]:
output_file = f'{OUTPUT_DIR}/pa3-{SAMPLE_ID}-Output.txt'
with open(output_file, 'w') as file:
    file.write(f"{num_samples}, {output_file}\n")
    for sample in range(num_samples):
        file.write(f"{d_k[sample][0]:.2f} {d_k[sample][1]:.2f} {d_k[sample][2]:.2f} ")
        file.write(f"{pt[sample][0]:.2f} {pt[sample][1]:.2f} {pt[sample][2]:.2f}")
        file.write(f" {dist[sample]:.2f}")
        file.write("\n")

print(f"Output written to {output_file}")


### compare experimental and expected output

In [177]:
import csv

output_csv = f'{OUTPUT_DIR}/error_analysis.csv'

with open(output_csv, 'w', newline='') as csvfile:
    fieldnames = ['Dataset', 'Pointer tip MAE', 'Closest point MAE', 'Error norm']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for SAMPLE_ID in ['A', 'B', 'C', 'D', 'E', 'F']:
        pred_file = f'{OUTPUT_DIR}/pa3-{SAMPLE_ID}-Output.txt'
        gt_file = f'{DATA_DIR}/PA3-{SAMPLE_ID}-Debug-Output.txt'

        if os.path.exists(pred_file) and os.path.exists(gt_file):
            matcher = FileOutputMatcher()

            error_d_k, error_c_k, error_norm = matcher(pred_file, gt_file)
            avg_error_d_k = round(np.mean(error_d_k), 3)
            avg_error_c_k = round(np.mean(error_c_k), 3)
            error_norm = round(error_norm, 3)

            writer.writerow({
                'Dataset': f'Debug {SAMPLE_ID}',
                'Pointer tip MAE': avg_error_d_k,
                'Closest point MAE': avg_error_c_k,
                'Error norm': error_norm
            })
        else:
            print(f'No prediction or ground-truth file found for sample {SAMPLE_ID}. Skipping operation.')

print(f"Error analysis written to {output_csv}")

Error analysis written to ./OUTPUT/error_analysis.csv


### unknown g, h, j summary

In [ ]:
input = f'{OUTPUT_DIR}/pa3-J-Output.txt'
output_csv = f'{OUTPUT_DIR}/J-summary.csv'

with open(input, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the first line
    d_x = []
    d_y = []
    d_z = []
    c_x = []
    c_y = []
    c_z = []
    norm = []
    for row in reader:
        row = list(map(float, row[0].split()))
        d_x.append(row[0])
        d_y.append(row[1])
        d_z.append(row[2])
        c_x.append(row[3])
        c_y.append(row[4])
        c_z.append(row[5])
        norm.append(row[6])

d_x_avg = np.mean(d_x)
d_y_avg = np.mean(d_y)
d_z_avg = np.mean(d_z)
c_x_avg = np.mean(c_x)
c_y_avg = np.mean(c_y)
c_z_avg = np.mean(c_z)
norm_avg = np.mean(norm)

with open(output_csv, 'w', newline='') as csvfile:
    fieldnames = ['Pointer tip average', 'Closest point average', 'Error norm average']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({
        'Pointer tip average': round(d_x_avg, 3),
        'Closest point average': round(c_x_avg, 3),
        'Error norm average': round(norm_avg, 3)
    })